# Lectura y preprocesamiento de documentos

In [1]:
!pip install --upgrade PyPDF2==2.12.1
!pip install os-sys

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached os_sys-2.1.4-py3-none-any.whl (15.6 MB)
  Using cached mysql-connector-2.2.9.tar.gz (11.9 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pyspeedtest-1.2.7.tar.gz (6.8 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached SQLAlchemy-1.3.24-cp39-cp39-win_amd64.whl (1.2 MB)
  Using cached pyvalid-1.0.4-py3-none-any.whl (11 kB)
  Using cached Django-4.1.7-py3-none-any.whl (8.1 MB)
  Using cached os_sys-2.1.3-py3-none-any.whl (15.5 MB)
  Using cached os_sys-2.1.2-py3-none-any.whl (15.4 MB)
  Using cached os_sys-2.1.1-py3-none-any.whl (15.4 MB)
  Using cached os_sys-2.1.0-py3-none-any.whl (15.9 MB)
  Using cached os_sys-2.0.9-py3-none-any.whl (15.4 MB)
  Using cached os_sys-2.0.8-py3-n

ERROR: os-sys has an invalid wheel, os-sys has an invalid wheel, could not read 'os_sys-1.9.3.dist-info/WHEEL' file: KeyError("There is no item named 'os_sys-1.9.3.dist-info/WHEEL' in the archive")


In [2]:
# Paquetes necesarios

import re
import pandas as pd
import os
import PyPDF2
import time
import sys
import pickle

## Almacenar los documentos en una base de datos

In [3]:
#---------------------------------------
# Cargar cada pdf en una lista de Python
#---------------------------------------

# Directorio de trabajo absoluto
# Necesario para llamados relativos desde bucles
scriptPath = sys.path[0]
os.chdir(scriptPath + '/Last_conpes_pdfs')

# Directorio con los pdfs
files = os.listdir(scriptPath + '/Last_conpes_pdfs')


# Lista vacía para llenar con cada pdf almacenado
Database = []

# Bucle que abre cada pdf en Python
for FILE in files:
    # Si termina en '.pdf' leerlo en formato binario y pegarlo a la lista
    if FILE.endswith('.pdf'):
        data = open(FILE,'rb') 
        Database.append(data)

Observemos los primeros 10 archivos

In [4]:
Database[0:10]

[<_io.BufferedReader name='3873.pdf'>,
 <_io.BufferedReader name='3874.pdf'>,
 <_io.BufferedReader name='3875.pdf'>,
 <_io.BufferedReader name='3876.pdf'>,
 <_io.BufferedReader name='3877.pdf'>,
 <_io.BufferedReader name='3878.pdf'>,
 <_io.BufferedReader name='3879.pdf'>,
 <_io.BufferedReader name='3880.pdf'>,
 <_io.BufferedReader name='3881.pdf'>,
 <_io.BufferedReader name='3882.pdf'>]

## Leer los documentos con bucle

Con el paquete PyPDF2 podemos leer archivos en pdf con facilidad. Sin embargo, hay varios documentos que no se encuentran en formato legible, es decir que para ellos hay que emplear el procedimiento de Reconocimiento Óptico de Caracteres. En este caso obviaremos dichos documentos por ser minoría.

In [5]:
# Dataframe vacío que almacenará el nombre del PDF y su texto
text_table = pd.DataFrame(index = [0], columns = ['PDF','Text'])

fileIndex = 0

# Contabilizar tiempo de ejecución
t0 = time.time()

# Para cada pdf que puede ser leído
for file in files:
  # Leer como binario
  pdfFileObj = open(file,'rb')
  # Magia con PyPDF2
  pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
  # Empezar contador de páginas
  startPage = 0
  # Caracter vacío para rellenarse con texto
  text = ''
  cleanText = ''
  # Mientras que el contador sea menor al número de páginas
  while startPage <= pdfReader.numPages-1:
    pageObj = pdfReader.getPage(startPage)
    text += pageObj.extractText()
    startPage += 1
  pdfFileObj.close()
  # Para cada palabra dentro del texto
  for myWord in text:
    # Ignorar saltos de línea
    if myWord != '\n':
      cleanText += myWord
  # Objeto temporal que contiene el texto limpio
  text = cleanText
  # Crear fila vacía
  newRow = pd.DataFrame(index = [0], columns = ['PDF', 'Text'])
  # Rellenar la anterior fila con nombre y texto
  newRow.iloc[0]['PDF'] = file
  newRow.iloc[0]['Text'] = text
  # Concatenar la fila creada al dataframe creado por fuera del bucle
  text_table = pd.concat([text_table, newRow], ignore_index=True)        


t1 = time.time()

# Tiempo total de ejecución
total = t1-t0

Observemos el dataframe creado

In [6]:
text_table = text_table.iloc[1:]
text_table

,PDF,Text
1,3873.pdf,Documento CONPES CONSEJO NAC...
2,3874.pdf,Documento CONPES CONSEJO NACI...
3,3875.pdf,Documento CONPES CONSEJO NAC...
4,3876.pdf,Documento CONPES CONSEJO NA...
5,3877.pdf,Documento CONPES CONSEJO NAC...
...,...,...
89,3965.pdf,1 Documento CONPES CONSEJO NAC...
90,3966.pdf,Documento CONPES CONSEJO NA...
91,3968.pdf,Documento CONPES CONSEJO NACI...
92,3969.pdf,Documento CONPES CONSEJO NA...


## Limpieza del texto

En procesamiento de lenguaje natural, para que una computadora pueda interpretar las palabras como números, es necesario realizar transformaciones sobre estas. En ese sentido, el texto se homogeniza retirando todos los elementos que no le aportan al verdadero significado del texto. Entre esos elementos encontramos:

- Caracteres especiales.
- Palabras mal escritas de corta longitud.
- Acentos si estos con inconsistentes (en español corregir la ortografía es más complicado que en inglés).
- Stop words: artículos, conectores y palabras que dependiendo del contexto se consideren stop words.

In [7]:
# Remover caracteres que no son ASCII
def remove_noChar(words):
    return [re.sub(u"[^a-zA-ZñÑáéíóúÁÉÍÓÚ ]","", word) for word in words]

# Remover stopwords
def remove_sw(words,sw_list):
    return [word for word in words if word not in sw_list]

# Remover caracteres cortos
def remove_shortW(words):
    return [word for word in words if len(word) > 2]

# Remover acentos
# Esto no es necesario para documentos que tienen buena ortografía
def remove_tilde(words):
    return [r_tilde(word) for word in words]

# Reemplazo de tildes
def r_tilde(word):
    w=[]
    for letra in list(word):
        if letra == 'á': letra = 'a'
        if letra == 'é': letra = 'e'
        if letra == 'í': letra = 'i'
        if letra == 'ó': letra = 'o'
        if letra == 'ú': letra = 'u'
        w += letra
    return ''.join(w)

# Función que usa las funciones anteriores
def preProc_docs(documentos):
    documentos = [re.sub(r'[^\w\s]',' ',proy) for proy in documentos]
    documentos = [[word for word in texto.lower().split()] for texto in documentos]
    documentos = [remove_noChar(text) for text in documentos]
    #documentos = [remove_tilde(text) for text in documentos]
    documentos = [remove_sw(words_lst, all_stopwords) for words_lst in documentos]
    documentos = [remove_shortW(text) for text in documentos]
    return [' '.join(item) for item in documentos]


In [10]:
nltk.download('stopwords')
import nltk

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hecto\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [13]:
# archivo stopwords
with open(scriptPath + '/stop_words_spanish.txt', 'rb') as f:
    sw_spanish = f.read().decode('latin-1').replace(u'\r', u'').split(u'\n')

sw_spanish

#---------------------------------------------------------------------------------

# conjunto de stopwords de nltk
default_stopwords = nltk.corpus.stopwords.words('spanish')

# Combinar ambos conjuntos de stopwords
all_stopwords = list(set(default_stopwords) | set(sw_spanish) )



all_stopwords[1:40]

['han',
 'si',
 'fuisteis',
 'vosotros',
 'míos',
 'estar',
 'yo',
 'seamos',
 'tendríais',
 'cierta',
 'algunas',
 'tenían',
 'solo',
 'pocos',
 'c',
 'eramos',
 'haya',
 'ningún',
 'estuvieron',
 'podemos',
 'ocho',
 'hizo',
 'de',
 'hubieras',
 'actualmente',
 'no',
 'teniendo',
 'estaremos',
 'desde',
 'alguno',
 'cual',
 'estará',
 'tampoco',
 'esto',
 'qeu',
 'estaré',
 'diferentes',
 'fuerais',
 'y',
 'n',
 'llegó',
 'habrían',
 'cuánta',
 'todavía',
 'tenidas',
 'era',
 'días',
 'hubiésemos',
 'fueses',
 'trata',
 'su',
 'hablan',
 'sus',
 'tuviera',
 'usted',
 'quién',
 'según',
 'fuera',
 'tendrán',
 'proximo',
 'asi',
 'haber',
 'cuántas',
 'debido',
 'cuales',
 'aquella',
 'dejó',
 'seáis',
 'embargo',
 'realizado',
 'segunda',
 'ésta',
 'éste',
 'le',
 'v',
 'lo',
 'm',
 'hecho',
 'hemos',
 'tuyo',
 'nuestros',
 'podriamos',
 'tiempo',
 'habían',
 'final',
 'cuatro',
 'pais',
 'mía',
 'p',
 'afirmó',
 'mayor',
 'conseguimos',
 'hubiese',
 'haceis',
 'trabajo',
 'encima',
 

Finalmente, con las funciones de limpieza listas, y la lista de stop words cargada, procedemos a limpiar el texto

### Correr limpieza

In [14]:
# Ejecutemos la limpieza y cronometremos el tiempo que toma correr

t0 = time.time()

clean_text = pd.DataFrame(text_table.Text)

clean_text = clean_text.apply(preProc_docs)

t1 = time.time()

total = t1-t0

## Extraer información relevante con expresión regular

In [15]:
#re.search(r'DEPARTAMENTO NACIONAL DE PLANEACIÓN\.(.*?)Departamento Nacional de Planeación', text_table.Text[5])
titles = []
for i in range(1,len(clean_text)):
    try:
        
        temp = re.search('nacional planeación(.*)versión aprobada', clean_text.Text.iloc[i])
        temp2 = temp.group(1).strip()
        temp3 = temp2.replace('CONSEJO NACIONAL DE POLÍTICA ECONÓMICA Y SOCIAL  REPÚBLICA DE COLOMBIA DEPARTAMENTO NACIONAL DE PLANEACIÓN       ','')
        titles.append(temp2)
        
    except:
        pass
    
    #except Exception:
    #    temp = re.search('Documento  CONPES(.*)Versión aprobada', text_table.Text[i])
    
    #if not os.path.isfile(filename):
    #    try: 
    #        urllib.request.urlretrieve(url, filename)
            
    #    except Exception:
    #        pass

len(titles)


74

In [23]:
titles[1:10]

['concepto favorable nación otorgar garantía fondo empresarial creado ley contratar operaciones pasivas crédito interno suma millones pesos departamento nacional planeación ministerio hacienda crédito público superintendencia servicios públicos domiciliarios',
 'concepto favorable nación contratar empréstitos externos libre destinación rápido desembolso líneas crédito contingentes ocurrencia desastres naturales entidades financieras internacionales organismos multilaterales entidades fomento gobiernos suma usd millones equivalente monedas financiar apropiaciones presupuestales prioritarias gobierno nacional departamento nacional planeación ministerio hacienda crédito público',
 'declaración importancia estratégica sistema identificación potenciales beneficiarios sisbén departamento nacional planeación',
 'distribución excedentes financieros establecimientos públicos empresas industriales comerciales societarias orden nacional corte diciembre modificación documento conpes departamento n

## Almacenar

In [16]:
pickle.dump(titles, open( "titles.p", "wb" ) )